In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
!pip install catboost
from catboost import *

Mounted at /content/drive
     |████████████████████████████████| 76.1 MB 57 kB/s 


In [2]:
!nvidia-smi

Fri Feb  4 19:26:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
with open('/content/drive/MyDrive/export_y.pickle', 'rb') as file:
    y_train = pickle.load(file)

In [4]:
with open('/content/drive/MyDrive/qifs.pickle', 'rb') as file:
    qifs = pickle.load(file)

In [5]:
with open('/content/drive/MyDrive/qifs.pickle', 'rb') as file:
    qifs = pickle.load(file)

In [6]:
X_train = pd.read_csv('/content/drive/MyDrive/export_train.csv')

In [7]:
pool = Pool(
    data=X_train.values,
    label=y_train/5,
    group_id=qifs
)

In [8]:
model_params = {
    'iterations': 18000,
    'custom_metric': ['NDCG:top=5'],
    'loss_function': 'YetiRank',  #Pairwise',
    'verbose': True,
    'random_seed': 1337,
    'max_bin' : 254,
    'task_type': 'GPU',
    'max_depth': 10,
    'use_best_model': False,
    'learning_rate': 0.15
}

In [9]:
model = CatBoost(model_params)
model.load_model("deep_yeti_trees.cbm",format="cbm")

In [ ]:
model = CatBoost(model_params)
model.fit(pool, eval_set=[pool])

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Выходные данные были обрезаны до нескольких последних строк (5000).
13002:	test: 0.9125714	best: 0.9125714 (13002)	total: 19m 7s	remaining: 7m 20s
13003:	test: 0.9125714	best: 0.9125714 (13003)	total: 19m 7s	remaining: 7m 20s
13004:	test: 0.9125714	best: 0.9125714 (13004)	total: 19m 7s	remaining: 7m 20s
13005:	test: 0.9125714	best: 0.9125714 (13004)	total: 19m 7s	remaining: 7m 20s
13006:	test: 0.9125714	best: 0.9125714 (13004)	total: 19m 7s	remaining: 7m 20s
13007:	test: 0.9125714	best: 0.9125714 (13007)	total: 19m 7s	remaining: 7m 20s
13008:	test: 0.9125714	best: 0.9125714 (13007)	total: 19m 7s	remaining: 7m 20s
13009:	test: 0.9125714	best: 0.9125714 (13009)	total: 19m 7s	remaining: 7m 20s
13010:	test: 0.9125714	best: 0.9125714 (13010)	total: 19m 7s	remaining: 7m 20s
13011:	test: 0.9125714	best: 0.9125714 (13011)	total: 19m 7s	remaining: 7m 20s
13012:	test: 0.9125714	best: 0.9125714 (13011)	total: 19m 8s	remaining: 7m 19s
13013:	test: 0.9125714	best: 0.9125714 (13011)	total: 19m 8s	re

In [ ]:
#model.save_model("deep_yeti_trees.cbm",format="cbm")

In [10]:
X_test = pd.read_csv("/content/drive/MyDrive/export_test.csv")

In [11]:
preds = []
pred_ids = list(set(X_test['QueryId'].values))
for elem in tqdm(pred_ids):
    tmp = X_test[X_test.QueryId==elem].drop(columns=['QueryId'])
    doc_ids = tmp['DocumentId'].values
    tmp = tmp.drop(columns=['DocumentId'])
    tmp = Pool(data=tmp.values)
    preds.extend(list(model.predict(tmp, ntree_end=6000)))

100%|██████████| 4200/4200 [00:47<00:00, 88.08it/s]


In [12]:
df_example = pd.read_csv('/content/drive/MyDrive/sample.csv')

In [13]:
df_example = df_example.sort_values(by=['QueryId','DocumentId']).reset_index().drop(columns=['index'])

In [14]:
df_example['preds'] = preds

In [15]:
df_example = df_example.sort_values(by=['QueryId','preds'], ascending=[True,False]).reset_index().drop(columns=['index'])

In [16]:
df_example = df_example.drop(columns=['preds'])

In [17]:
df_example.to_csv("prediction_yeti6000.csv", index=False)